In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def func(x):
    return np.exp(-2*x)*np.cos(10*x)

def func_integral(x):
    return (-1/52)*np.exp(-2*x)*(np.cos(10*x)-5*np.sin(10*x))

Define Trapezoidal Method

In [ ]:
# Core

def trapezoid_core(f,x,h):
    return 0.5*h*(f(x+h) + f(x))

# Define a wrapper function to perform Simpson's method

def trapezoid_method(f,a,b,N):
    # f == function to integrate
    # a == lower limit f integration
    # b = upper limit of integration
    # N == number of function evaluations to use
    # number of chunks will be N-1, so if N is odd, then we don't need to adjust the last segment
    
    # define x values to perform trapezoidal rule
    x = np.linspace(a,b,N)
    h = x[1] - x[0]
    
    # define the value of the integral
    Fint = 0.0
    
    # perform the integral using trapezoidal rule
    for i in range(0, len(x) - 1,1):
        Fint += trapezoid_core(f,x[i],h)
        
    return Fint

Define Simpson's Method

In [ ]:
# Core

def simpson_core(f,x,h):
    return h*(f(x) + 4*f(x+h) + f(x+2*h))/3

# Define a wrapper function to perform Simpson's method

def simpson_method(f,a,b,N):
    
    x = np.linspace(a,b,N)
    h = x[1] - x[0]
    
    Fint = 0.0
    
    # if n is odd or even, we have different number of chunks to add
    
    if ((N%2) == 0):
        lim = 3
    else:
        lim = 2
    
    #perform the integral using Simpson's method
    
    for i in range(0, len(x)-2, 2):
        Fint += simpson_core(f,x[i],h)
        
    if ((N%2) == 0):
        Fint += simpson_core(f,x[-2],0.5*h)
        
    return Fint
    

Romberg Integration

In [ ]:
# Core

def romberg_core(f,a,b,i):
    
    #we need the difference b-a ()
    h = b - a
    
    #and the increment between the new function evaluations
    dh = h/2.**(i)
    
    #we need the cofactor
    K = h/2.**(i+1)
    
    #and the function evaluations
    M = 0.0
    for j in range(2**i):
        M += f(a + 0.5*dh + j*dh)
        
    return K*M

# Define a wrapper function to perform Romberg's method

def romberg_integration(f,a,b,tol):
    
    # define an iteration variable
    i = 0
    
    #define max number of iterations
    imax = 1000
    
    #define an error estimate, set to a large value
    delta = 100.0*np.fabs(tol)
    
    #set an array of integral answers
    I = np.zeros(imax, dtype = float)
    
    #get the zerothromberg iteration
    I[0] = 0.5*(b-a)*(f(a) + f(b))
    
    #iterate by 1
    i += 1
    
    while (delta>tol):
        
        #find this romberg iteration
        I[i] = 0.5*I[i-1] + romberg_core(f,a,b,i)
        
        #compute the new fractional error estimate
        delta = np.fabs((I[i] - I[i-1])/I[i])
        
        print(i, I[i], I[i-1], delta)
        
        if (delta>tol):
            
            #iterate 
            i += 1
            
            #if we've reached the maximum iterations
            if (i>imax):
                print('Max iterations reached')
                raise StopIteration('Stopping iterations after', i)
                
    return I[i]

Find integrals

In [ ]:
answer = func_integral(np.pi) - func_integral(0)
print('Answer:', answer)

print('Trapezoid')
print(trapezoid_method(func,0,np.pi,N_trap))

print('Simpson')
print(simpson_method(func,0,np.pi,N_simp))

print('Romberg')
tolerance = 1.0e-6
RI = romberg_integration(func,0,np.pi,tolerance)
print(RI, (RI-answer)/answer, tolerance)

Find number of intervals needed for specified accuracy in Trapezoidal and Simpson's methods:

In [ ]:
y = np.linspace(0,np.pi,1000)
def deriv2(x):
    return (-96*np.exp(-2*x)*np.cos(10*x)) + (40*np.exp(-2*x)*np.sin(10*x))

def deriv4(x):
    return (7616*np.exp(-2*x)*np.cos(10*x)) - (7680*np.exp(-2*x)*np.sin(10*x))

'''
I use the formulas for error bounds in Trapezoidal and Simpson's methods:
    E_trapeizoidal = ((b-a)^3*deriv2(max))/(12*N^2)
    E_simpson = ((b-a)^5*deriv2(max))/(180*N^4)
'''

values2 = []
values4 = []

for i in y:
    values2.append(deriv2(i))
    values4.append(deriv4(i))
    
max2 = np.max(values2)
max4 = np.max(values4)

N_trap = (((np.pi)**3*max2)/(12*tolerance))**(1/2)
N_simp = (((np.pi)**5*max2)/(180*tolerance))**(1/4)

print('Intervals needed for specified accuracy in Trapezoidal Method:', int(np.ceil(N_trap)))
print("Intervals needed for specified accuracy in Simpson's Method: ", int(np.ceil(N_simp)))